## Introduction to the GLM

Now that the data is clean we can perform analysis. Resting state analysis is completed in notebook 6. This is a review of the general linear model using data that was collected during a breathholding task. 

### Step 1: Ensure you have data from the breath-hold task downloaded to your local machine. 
    
    #TODO: link to download
    
### Step 2: We need to figure out the stimulus for the breath holding task. 

The description of this task is contained in `data/ds000030/task-bht_bold.json`. Read through this folder and determine if you can create an array of timings where the subject was and was not holding their breath.

In [15]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

TR = 2
scan_length = 166

act_length = 13.5
rest_length = 13.5
n_blocks = scan_length/(rest_length + act_length)

volumes = np.linspace(1, scan_length-1, scan_length/TR) 



/Users/oliviastanley/miniconda3/envs/conpcourse/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  if sys.path[0] == '':


### Step 3: Now that we know what stimulus was used we need to convolve it with a hemodynamic response function. 

This can be done using using the nitime.fmri.hrf function. 

    #TODO: Full guide

### Step 4: Now we can fit our model of what task was acquired to our input data

This can be done using the permuted ols function from nilearn. We can also include motion confounds as before simulataneously in order to remove them during fitting opposed to cleaning the data. This is the standard for task based data.